In [1]:
import sys
sys.path.append('C:/Users/reese/OneDrive/Documents/SNHU/CS340')
%pip install jupyter-dash
%pip install dash-leaflet
%pip install pymongo
%pip install pandas

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
# Setup the Jupyter version of Dash
from dash import Dash

# Configure the necessary Python module imports for dashboard components
import dash
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State
import base64

# Configure OS routines
import os

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re #needed for the regex pattern matching

#### FIX ME #####
# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
from crud_operations_enhancement import CRUD

###########################
# Data Manipulation / Model
###########################
# FIX ME update with your username and password and CRUD Python module name

username = "aacuser"
password = "SNHU1234"
host = "localhost"
port = 32824
database_name = "AAC"
collection_name = "animals"

# Connect to database via CRUD Module
crud = CRUD(database_name, collection_name)

# class read method must support return of list object and accept projection json input
# sending the read method an empty document requests all documents be returned
df = pd.DataFrame.from_records(crud.read({}))

# MongoDB v5+ is going to return the '_id' column and that is going to have an 
# invlaid object type of 'ObjectID' - which will cause the data_table to crash - so we remove
# it in the dataframe here. The df.drop command allows us to drop the column. If we do not set
# inplace=True - it will reeturn a new dataframe that does not contain the dropped column(s)
df.drop(columns=['_id'],inplace=True)

## Debug
## print(len(df.to_dict(orient='records')))
## print(df.columns)

###########################
# Function to Fetch Data
###########################
def fetch_data(filter_option):
    """Fetch and filter data based on rescue type."""
    query = {}
    if filter_option == "water":
        query = {"breed": {"$in": [re.compile("lab", re.IGNORECASE), re.compile("chesa", re.IGNORECASE), re.compile("newf", re.IGNORECASE)]},
                 "age_upon_outcome_in_weeks": {"$gte": 26}}
    elif filter_option == "mountain":
        query = {"breed": {"$in": [re.compile("german", re.IGNORECASE), re.compile("mala", re.IGNORECASE), re.compile("old english sheep", re.IGNORECASE), re.compile("husk", re.IGNORECASE), re.compile("rott", re.IGNORECASE)]},
                 "age_upon_outcome_in_weeks": {"$gte": 26}}
    elif filter_option == "disaster":
        query = {"breed": {"$in": [re.compile("german", re.IGNORECASE), re.compile("golden", re.IGNORECASE), re.compile("blood", re.IGNORECASE), re.compile("dober", re.IGNORECASE), re.compile("rott", re.IGNORECASE)]},
                 "age_upon_outcome_in_weeks": {"$gte": 20, "$lte": 300}}
    # Reset returns all data
    records = crud.read(query)
    return pd.DataFrame.from_records(records).drop(columns=['_id'], errors='ignore')

###########################
# Load Initial Data
###########################
df = fetch_data("reset")

#########################
# Dashboard Layout / View
#########################
app = Dash(__name__)

#FIX ME Add in Grazioso Salvare’s logo
image_filename = 'Grazioso Salvare Logo.png' 
if not os.path.exists(image_filename):
    raise FileNotFoundError(f"Logo file '{image_filename}' not found.")
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

# Add a dictionary to simulate user-specific data filters
USER_FILTERS = {
    "user1": {"breed": {"$regex": "lab", "$options": "i"}},
    "user2": {"breed": {"$regex": "german", "$options": "i"}},
    "admin": {}  # Admin sees all data
}

#FIX ME Place the HTML image tag in the line below into the app.layout code according to your design
#FIX ME Also remember to include a unique identifier such as your name or date
# html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()))

# Add a login form to the layout
app.layout = html.Div(
    id='page-content',
    style={'backgroundColor': 'white', 'color': 'black'},  # Default light theme
    children=[
        html.Center(
            html.A(
                href="https://www.snhu.edu",
                children=html.Img(
                    src='data:image/png;base64,{}'.format(encoded_image.decode()),
                    style={'width': '200px'}
                )
            )
        ),
        html.Center(html.B(html.H1('Reese\'s CS-340 Dashboard'))),
        html.Hr(),
        html.Div([
        #FIXME Add in code for the interactive filtering options. For example, Radio buttons, drop down, checkboxes, etc.
        dcc.RadioItems(
            id='filter-options',
            options=[
                {'label': 'Water Rescue', 'value': 'water'},
                {'label': 'Mountain or Wilderness Rescue', 'value': 'mountain'},
                {'label': 'Disaster or Individual Tracking', 'value': 'disaster'},
                {'label': 'Reset', 'value': 'reset'}
            ],
            value='reset',
            labelStyle={'display': 'inline-block', 'margin-right': '10px'}
        )]),
        html.Hr(),
        html.Div([
            html.Label("Username:"),
            dcc.Input(id='username', type='text', placeholder='Enter username', style={'margin-right': '10px'}),
            html.Label("Password:"),
            dcc.Input(id='password', type='password', placeholder='Enter password', style={'margin-right': '10px'}),
            html.Button('Login', id='login-button', n_clicks=0),
            html.Div(id='login-message', style={'color': 'red', 'margin-top': '10px'})
        ]),
        html.Hr(),
        html.Div(id='dashboard-content', style={'display': 'none'})  # Hidden until login
    ]
)

@app.callback(
    [Output('dashboard-content', 'style'),
     Output('datatable-id', 'data'),
     Output('login-message', 'children')],
    [Input('login-button', 'n_clicks'),
     Input('filter-options', 'value')],
    [State('username', 'value'),
     State('password', 'value')]
)
def handle_login_and_filter(n_clicks, filter_option, username, password):
    # Use dash.callback_context to determine which input triggered the callback
    ctx = dash.callback_context

    # Determine which input triggered the callback
    if not ctx.triggered:
        return {'display': 'none'}, [], ""

    triggered_id = ctx.triggered[0]['prop_id'].split('.')[0]

    # Handle login logic
    if triggered_id == 'login-button':
        if n_clicks > 0:
            if username in USER_FILTERS and password == "password123":  # Replace with secure authentication logic
                # Fetch user-specific data
                user_filter = USER_FILTERS[username]
                dff = pd.DataFrame.from_records(crud.read(user_filter)).drop(columns=['_id'], errors='ignore')
                return {'display': 'block'}, dff.to_dict('records'), ""
            else:
                return {'display': 'none'}, [], "Invalid username or password. Please try again."

    # Handle filtering logic
    elif triggered_id == 'filter-options':
        dff = fetch_data(filter_option)
        return {'display': 'block'}, dff.to_dict('records'), ""

    # Default return
    return {'display': 'none'}, [], ""

# Add the dashboard content (hidden by default)
dashboard_content = html.Div([
    html.Div([
        html.Label('Select Theme:'),
        dcc.RadioItems(
            id='theme-toggle',
            options=[
                {'label': 'Light', 'value': 'light'},
                {'label': 'Dark', 'value': 'dark'}
            ],
            value='light',
            labelStyle={'display': 'inline-block', 'margin-right': '10px'}
        )
    ]),
    html.Hr(),
    dash_table.DataTable(
        id='datatable-id',
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
        data=[],  # Data will be populated after login
        editable=False,
        filter_action="native",
        sort_action="native",
        sort_mode="multi",
        row_selectable="single",
        selected_rows=[],
        page_action="native",
        page_size=10,
        style_table={'overflowX': 'auto'},
        style_data={},  # Will be dynamically styled
        style_header={},  # Will be dynamically styled
    ),
    html.Br(),
    html.Hr(),
    html.Div(
        id='modal',
        style={
            'display': 'none',  # Hidden by default
            'position': 'fixed',
            'zIndex': 1,
            'left': 0,
            'top': 0,
            'width': '100%',
            'height': '100%',
            'overflow': 'auto',
            'backgroundColor': 'rgba(0,0,0,0.4)',
            'paddingTop': '50px'
        },
        children=[
            html.Div(
                style={
                    'backgroundColor': '#fefefe',
                    'margin': '5% auto',
                    'padding': '20px',
                    'border': '1px solid #888',
                    'width': '80%'
                },
                children=[
                    html.Span(
                        '×',
                        id='close-modal',
                        style={
                            'color': '#aaa',
                            'float': 'right',
                            'fontSize': '28px',
                            'fontWeight': 'bold',
                            'cursor': 'pointer'
                        }
                    ),
                    html.Div(id='modal-content')  # Content will be dynamically updated
                ]
            )
        ]
    ),
    html.Hr(),
    html.Br(),
    html.Div([
        html.Label("Choose a feature to visualize:"),
        dcc.Dropdown(
            id='pie-feature-dropdown',
            options=[
                {'label': 'Breed', 'value': 'breed'},
                {'label': 'Sex', 'value': 'sex_upon_outcome'},
                {'label': 'Outcome Type', 'value': 'outcome_type'}
            ],
            value='breed'  # default
        ),
        html.Br(),
        html.Button("Download Data", id="download-button"),
        dcc.Download(id="download-dataframe-csv"),
        html.Br(),
        html.Label("Select Chart Type:"),
        dcc.RadioItems(
            id='chart-type',
            options=[
                {'label': 'Pie Chart', 'value': 'pie'},
                {'label': 'Bar Chart', 'value': 'bar'},
                {'label': 'Donut Chart', 'value': 'donut'}
            ],
            value='pie',
            labelStyle={'display': 'inline-block'}
        )
    ]),
    html.Hr(),
    html.Label("Select Map Layer:"),
    dcc.Dropdown(
    id='map-layer',
    options=[
        {'label': 'Streets', 'value': 'streets'},
        {'label': 'Satellite', 'value': 'satellite'}
    ],
    value='streets'
    ),
    html.Div(className='row',
             style={'display': 'flex'},
             children=[
                 html.Div(
                     id='graph-id',
                     className='col s12 m6',
                 ),
                 html.Div(
                     id='map-id',
                     className='col s12 m6',
                 )
             ])
    ])

# Add the dashboard content to the layout
app.layout.children.append(dashboard_content)

#############################################
# Interaction Between Components / Controller
#############################################

@app.callback(
    [Output('modal', 'style'),
     Output('modal-content', 'children')],
    [Input('datatable-id', 'derived_virtual_selected_rows'),
     Input('close-modal', 'n_clicks')],
    [State('datatable-id', 'derived_virtual_data')]
)
def handle_modal(selected_rows, close_clicks, data):
    ctx = dash.callback_context

    # Determine which input triggered the callback
    if not ctx.triggered:
        return {'display': 'none'}, []

    triggered_id = ctx.triggered[0]['prop_id'].split('.')[0]

    # Close the modal if the close button is clicked
    if triggered_id == 'close-modal' and close_clicks:
        return {'display': 'none'}, []

    # Open the modal if a row is selected
    if triggered_id == 'datatable-id' and selected_rows and data:
        # Get the selected row's data
        row = data[selected_rows[0]]

        # Extract details and image URL (replace 'image_url' with the actual column name)
        name = row.get('name', 'Unknown')
        breed = row.get('breed', 'Unknown')
        age = row.get('age_upon_outcome', 'Unknown')
        image_url = row.get('image_url', '/assets/german-sheperd.jpg') 

        # Create modal content
        content = [
            html.H2(f"Details for {name}"),
            html.Img(src=image_url, style={'width': '300px', 'height': '300px'}),
            html.P(f"Breed: {breed}"),
            html.P(f"Age: {age}")
        ]

        # Show modal
        return {'display': 'block'}, content

    # Default return (hide modal)
    return {'display': 'none'}, []

@app.callback(
    Output("download-dataframe-csv", "data"),
    Input("download-button", "n_clicks"),
    State("datatable-id", "data")
)
def download_data(n_clicks, data):
    if n_clicks:
        dff = pd.DataFrame(data)
        return dcc.send_data_frame(dff.to_csv, "filtered_data.csv")

# Display the breeds of animal based on quantity represented in
# the data table
# Update the graph based on the data table
@app.callback(
    Output('graph-id', "children"),
    [
        Input('datatable-id', "derived_virtual_data"),
        Input('pie-feature-dropdown', 'value'),
        Input('chart-type', 'value')
    ]
)
def update_graphs(viewData, selected_feature, chart_type):
    dff = pd.DataFrame(viewData)
    if dff.empty or selected_feature not in dff.columns:
        return [html.P("No data available or feature not found.")]

    # Generate the selected chart type
    if chart_type == 'pie':
        fig = px.pie(dff, names=selected_feature, title=f"Distribution of {selected_feature}")
    elif chart_type == 'bar':
        fig = px.bar(dff, x=selected_feature, title=f"Distribution of {selected_feature}", color_discrete_sequence=["#636EFA"])
    elif chart_type == 'donut':
        fig = px.pie(dff, names=selected_feature, title=f"Distribution of {selected_feature}", hole=0.4)

    # Update layout for better visualization
    fig.update_layout(
        legend_title_text=selected_feature.capitalize(),
        legend=dict(
            orientation="v",
            x=1.05,
            y=5,
            traceorder="normal",
            font=dict(size=12),
            bgcolor="rgba(0,0,0,0)"
        )
    )

    return [
        dcc.Graph(
            figure=fig
        )
    ]

@app.callback(
Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    if selected_columns is None:
        return []  # Return an empty list if nothing is selected
    return [{
        'if': {'column_id': i},
        'background_color': '#D2F3FF'
    } for i in selected_columns]

# This callback will update the geo-location chart for the selected data entry
# derived_virtual_data will be the set of data available from the datatable in the form of 
# a dictionary.
# derived_virtual_selected_rows will be the selected row(s) in the table in the form of
# a list. For this application, we are only permitting single row selection so there is only
# one value in the list.
# The iloc method allows for a row, column notation to pull data from the datatable
@app.callback(
    Output('map-id', "children"),
    [
        Input('datatable-id', "derived_virtual_data"),
        Input('datatable-id', "derived_virtual_selected_rows"),
        Input('map-layer', "value")  # Add map layer as an input
    ]
)
def update_map(viewData, index, map_layer):
    # Default Austin Animal Shelter location and details
    default_lat = 30.75
    default_long = -97.48
    default_tooltip = "Austin Animal Center"
    default_heading = "Austin Animal Center"
    default_paragraph = "Shelter Home Location"

    # Map tile URLs for different layers
    tile_urls = {
        'streets': "https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png",
        'satellite': "https://{s}.tile.opentopomap.org/{z}/{x}/{y}.png",
    }

    # Build default map
    base_map = [
        dl.Map(
            style={'width': '1000px', 'height': '500px'},
            center=[default_lat, default_long],
            zoom=10,
            children=[
                dl.TileLayer(url=tile_urls.get(map_layer, tile_urls['streets']), id="base-layer-id")
            ]
        )
    ]

    # Safeguard: Check if viewData is valid
    if not viewData or not isinstance(viewData, list):
        base_map[0].children.append(
            dl.Marker(
                position=[default_lat, default_long],
                children=[
                    dl.Tooltip(default_tooltip),
                    dl.Popup([
                        html.H1(default_heading),
                        html.P(default_paragraph)
                    ])
                ]
            )
        )
        return base_map

    # Convert viewData to DataFrame
    try:
        dff = pd.DataFrame(viewData)
    except Exception as e:
        print(f"Error converting viewData to DataFrame: {e}")
        return base_map

    # Safeguard: Check if index is valid
    if not index or not isinstance(index, list) or index[0] < 0 or index[0] >= len(dff):
        return base_map

    # Extract latitude, longitude, and additional details for the selected row
    row = index[0]
    latitude = dff.get('location_lat', [default_lat])[row]
    longitude = dff.get('location_long', [default_long])[row]
    tooltip = dff.get('name', [default_tooltip])[row]
    popup_heading = "Animal Details"
    popup_paragraph = f"Breed: {dff.get('breed', [default_paragraph])[row]}"

    # Add the selected row's marker to the base map
    base_map[0].children.append(
        dl.Marker(
            position=[latitude, longitude],
            children=[
                dl.Tooltip(tooltip),
                dl.Popup([
                    html.H1(popup_heading),
                    html.P(popup_paragraph)
                ])
            ]
        )
    )

    return base_map

# Callback to update the theme dynamically
@app.callback(
    [Output('page-content', 'style'),
     Output('datatable-id', 'style_data'),
     Output('datatable-id', 'style_header')],
    Input('theme-toggle', 'value')
)
def update_theme(theme):
    if theme == 'dark':
        page_style = {'backgroundColor': '#2c2c2c', 'color': 'white'}
        data_style = {'backgroundColor': '#2c2c2c', 'color': 'white'}
        header_style = {'backgroundColor': '#1a1a1a', 'color': 'white', 'fontWeight': 'bold'}
    else:
        page_style = {'backgroundColor': 'white', 'color': 'black'}
        data_style = {'backgroundColor': 'white', 'color': 'black'}
        header_style = {'backgroundColor': '#f2f2f2', 'color': 'black', 'fontWeight': 'bold'}
    return page_style, data_style, header_style



while True:
    try:
        app.run(debug=True, port=port)
        break
    except OSError:
        print(f"Port {port} in use. Trying next port...")
        port += 1

# Dash app running on http://127.0.0.1:32824/

Connected to MongoDB.
